In [ ]:
from sagemaker.predictor import RealTimePredictor, json_deserializer
import sagemaker
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.sklearn import SKLearn
import boto3
import json

# Get and print execution role

In [ ]:
role = sagemaker.get_execution_role()

In [ ]:
role

# Fit and Deploy Model

In [ ]:
class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=None, 
                         deserializer=json_deserializer, content_type='application/json')
        
model = PyTorch(entry_point='serve.py', name="twitter-fake-true-torch", role=role,
                framework_version='1.7',py_version='py36', source_dir='mysrc', instance_count=1, instance_type='ml.m5.4xlarge', 
                predictor_cls=Predictor)

In [ ]:
model.fit()

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.4xlarge')

#  Test The endpoint

In [ ]:

client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    Body=json.dumps({"text": "preved medved"}),
    ContentType='application/json',
    Accept="application/json"
)

In [ ]:
result = response['Body'].read().decode()

In [ ]:
print(json.loads(json.loads(result))['proba'])